In [97]:
import pprint
from collections import namedtuple

import matplotlib.pyplot as plt
import pandas as pd
import tqdm
from kmodes.kmodes import KModes

%matplotlib inline

In [98]:
Summary = namedtuple(
    "Summary",
    [
        "name",
        "num_rows",
        "num_cols",
        "num_classes",
        "missing_values",
#         "duplicate_rows",
        "adjusted_num_rows"
    ]
)

summaries = []
for dataset_name in ("breast_cancer", "mushroom", "soybean", "zoo"):

    df = pd.read_csv(f"../data/{dataset_name}.csv", na_values=["?", "dna"])
    
    classes = df["class"]
    df = df.drop("class", axis=1)

    dropped_df = df.dropna()
    missing_values = len(dropped_df) != len(df)
#     dropped_df = dropped_df.drop_duplicates()
#     duplicate_rows = len(dropped_df) != len(df)

    nrows, ncols = df.shape
    nclasses = classes.nunique()
    adjusted_nrows = len(dropped_df)
    
    summaries.append(
        Summary(
            dataset_name,
            nrows,
            ncols,
            nclasses,
            missing_values,
#             duplicate_rows,
            adjusted_nrows
        )
    )


In [99]:
summary_df = pd.DataFrame(summaries).set_index("name")
summary_df.index = [" ".join(name.split("_")).capitalize() for name in summary_df.index]
summary_df.columns = [" ".join(col.replace("num", "no.").split("_")).capitalize() for col in summary_df.columns]

summary_df.to_latex("../tex/dataset_summary.tex")
summary_df

,No. rows,No. cols,No. classes,Missing values,Adjusted no. rows
Breast cancer,699,9,2,True,683
Mushroom,8124,22,2,True,5644
Soybean,307,35,19,True,266
Zoo,101,17,7,False,101


In [87]:
dataset_costs = {}
for dataset_name in ("breast_cancer", "mushroom", "soybean", "zoo"):

    df = pd.read_csv(f"../data/{dataset_name}.csv", na_values=["?", "dna"])
    df = df.drop("class", axis=1).dropna().drop_duplicates()

    costs = []
    for k in tqdm.tqdm(range(2, 16)):
        km = KModes(n_clusters=k, init="random", n_init=10, random_state=0, n_jobs=1).fit(df)
        costs.append(km.cost_)

    dataset_costs[dataset_name] = costs



100%|██████████| 14/14 [00:10<00:00,  1.31it/s]

100%|██████████| 14/14 [03:20<00:00, 14.33s/it]

100%|██████████| 14/14 [00:14<00:00,  1.06s/it]

100%|██████████| 14/14 [00:03<00:00,  4.11it/s]


In [105]:
pprint.pprint(dataset_costs)

{'breast_cancer': [2128.0,
                   2038.0,
                   1956.0,
                   1877.0,
                   1846.0,
                   1804.0,
                   1777.0,
                   1752.0,
                   1716.0,
                   1702.0,
                   1683.0,
                   1646.0,
                   1638.0,
                   1619.0],
 'mushroom': [37662.0,
              32663.0,
              29645.0,
              27837.0,
              26716.0,
              25673.0,
              25376.0,
              24534.0,
              23803.0,
              22700.0,
              22375.0,
              21814.0,
              21179.0,
              20909.0],
 'soybean': [2170.0,
             1996.0,
             1886.0,
             1749.0,
             1667.0,
             1606.0,
             1548.0,
             1517.0,
             1466.0,
             1388.0,
             1388.0,
             1330.0,
             1321.0,
             1306.0],
 'z